In [6]:
# Working directory set
import sys
sys.path.append("../scripts/")

In [8]:
import pandas as pd
import numpy as np
import json
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import KFold

from clean_data import load_and_clean_data

pd.set_option('display.max_rows', 400)

In [3]:
data = load_and_clean_data()

/var/folders/dp/df65_q2x4lv3t41057t55p780000gn/T/ipykernel_65506/3315705005.py:1: DtypeWarning: Columns (10,17,19,23,25,27,28,29,30,32,33,36,37,38,39,40,41,42,43,50,51,83,86,89,92,94,95,97,100,103,106,110,113,116,121,124,127,130,133,136,188,190,201,212,231,232,242,244,247,249,256,262,267,273,279,280,281,282,283,284,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,305,318,320,333,334) have mixed types.Specify dtype option on import or set low_memory=False.
  data = load_and_clean_data()
/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureW

In [4]:
data.shape

(222647, 291)

In [14]:
# Baseline model
def run_baseline_dt(X_train, y_train, X_val, y_val):
    dt = DecisionTreeClassifier()
    dt.fit(X_train, y_train)
    y_pred = dt.predict_proba(X_val)[:,1]
    
    fpr, tpr, thresholds = roc_curve(y_val, y_pred)
    roc_auc = auc(fpr, tpr)
    
    importances = {}
    for name, importance in zip(X_val.columns, dt.feature_importances_):
        importances[name] = importance
        
    imp = pd.DataFrame.from_dict(importances, orient = 'index', columns = ['importance'])
    imp.sort_values(by = 'importance', ascending = False, inplace = True)
    
    return fpr, tpr, roc_auc, imp

In [9]:
X = data.drop('target', axis = 1)
y = data['target']

# 70 20 10 split
X_data, X_holdout, y_data, y_holdout = train_test_split(X, y, test_size = .1, random_state = 42)

In [12]:
kf = KFold(n_splits=10, shuffle=True, random_state=73)
auc_scores = []

for train, val in kf.split(X_data):
    X_train, y_train, X_val, y_val = X_data.iloc[train], y_data.iloc[train], X_data.iloc[val], y_data.iloc[val]
    _, _, roc_auc, _ = run_baseline_dt(X_train, y_train, X_val, y_val)
    auc_scores.append(roc_auc)

In [16]:
print(f'Mean CV AUC Score: {np.mean(auc_scores)}')

Mean CV AUC Score: 0.7207422535363419
